<a href="https://colab.research.google.com/github/ansonkwokth/PlackettLuceModel/blob/main/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !git clone https://github.com/ansonkwokth/PlackettLuceModel.git

In [2]:
# !python -m unittest plackett_luce/tests/test_utils.py

In [11]:
from plackett_luce import datasets as ds
from plackett_luce.model import PlackettLuceModel
from plackett_luce.utils import EarlyStopper

import torch
from torch import nn

torch.manual_seed(0)


In [12]:
# Custom neural network model for flexible scoring
class NaiveNN(nn.Module):
    def __init__(self, input_dim):
        super(NaiveNN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 1)  # 1D output for scoring
        )

    def forward(self, x):
        return self.network(x)





# Custom neural network model for flexible scoring
class LessNaiveNN(nn.Module):
    def __init__(self, input_dim):
        super(LessNaiveNN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 4),
            nn.ReLU(),
            nn.Linear(4, 1)  # 1D output for scoring
        )

    def forward(self, x):
        return self.network(x)


In [75]:
# Parameters
num_samples_train = 1000
num_samples_test = 1000
num_items = 10

# Data generation
print("Generating training and testing data...")
X_train, rankings_train = ds.generate_data(num_samples_train, num_items)
X_test, rankings_test = ds.generate_data(num_samples_test, num_items)
num_features = X_train.shape[-1]

# Create item masks for variable item counts
item_mask_train = torch.ones((num_samples_train, num_items))
item_mask_test = torch.ones((num_samples_test, num_items))
# Simulate some instances with fewer items (e.g., 5 items max but some with only 3)
# item_mask_train[torch.rand(num_samples_train, num_items) < 0.2] = 0  # Randomly mask some items
# item_mask_test[torch.rand(num_samples_test, num_items) < 0.2] = 0


Generating training and testing data...


In [76]:

# Initialize the model
# custom_nn = NaiveNN(input_dim=num_features)
custom_nn = LessNaiveNN(input_dim=num_features)
# Custom early stopper
custom_early_stopper = EarlyStopper(patience=20, min_delta=0.01)
model = PlackettLuceModel(score_model=custom_nn, early_stopper=custom_early_stopper)

# Training
print("Training the model...")

model.fit(X_train, rankings_train, lr=0.01, epochs=500, top_k=3)
# model.fit(X_train, rankings_train, lr=0.01, epochs=500)


Training the model...
Epoch 10/500, Negative Log-Likelihood: 6.3865
Epoch 20/500, Negative Log-Likelihood: 5.9553
Epoch 30/500, Negative Log-Likelihood: 5.0954
Epoch 40/500, Negative Log-Likelihood: 4.0044
Epoch 50/500, Negative Log-Likelihood: 3.1672
Epoch 60/500, Negative Log-Likelihood: 2.8472
Epoch 70/500, Negative Log-Likelihood: 2.7122
Epoch 80/500, Negative Log-Likelihood: 2.6228
Epoch 90/500, Negative Log-Likelihood: 2.5840
Epoch 100/500, Negative Log-Likelihood: 2.5569
Epoch 110/500, Negative Log-Likelihood: 2.5406
Epoch 120/500, Negative Log-Likelihood: 2.5256
Epoch 130/500, Negative Log-Likelihood: 2.5109
Epoch 140/500, Negative Log-Likelihood: 2.4951
Epoch 150/500, Negative Log-Likelihood: 2.4822
Early stopping at epoch 159 with NLL 2.4750


In [83]:

# Test the model
print("\nTesting the model...\n")
predicted_rankings = model.predict(X_test)

# Evaluate the performance
top1_correct = 0
top2_correct = 0
top1in3_correct = 0
top2in3_correct = 0
top1or2in3_correct = 0

print_first_few = 10
for i, (pred, true) in enumerate(zip(predicted_rankings, rankings_test.tolist())):
    if i < print_first_few:
        print(f"Sample {i + 1}:")
        print(f"  Predicted Ranking: {pred}")
        print(f"  True Ranking:      {true}")

    # Check Top-1 accuracy
    if pred[0] == true[0]:
        top1_correct += 1

    # Check Top-2 accuracy
    if pred[:2] == true[:2]:
        top2_correct += 1

    # Check Top-1 in first 3 accuracy
    if pred[0] in true[:3]:
        top1in3_correct += 1
    # Check Top-2 in first 3 accuracy
    if pred[1] in true[:3]:
        top2in3_correct += 1
    # Check Top-1 or 2 in first 3 accuracy
    if pred[0] in true[:3] or pred[1] in true[:3]:
        top1or2in3_correct += 1

# Compute percentages
top1_accuracy = top1_correct / num_samples_test * 100
top2_accuracy = top2_correct / num_samples_test * 100
top1in3_accuracy = top1in3_correct / num_samples_test * 100
top2in3_accuracy = top2in3_correct / num_samples_test * 100
top1or2in3_accuracy = top1or2in3_correct / num_samples_test * 100

print(f"\nTop-1 or 2 in 3 Accuracy: {top1or2in3_accuracy:.2f}%")
print(f"Top-1 in 3 Accuracy: {top1in3_accuracy:.2f}%")
print(f"Top-2 in 3 Accuracy: {top2in3_accuracy:.2f}%")
print(f"Top-1 Accuracy: {top1_accuracy:.2f}%")
print(f"Top-2 Accuracy: {top2_accuracy:.2f}%")



Testing the model...

Sample 1:
  Predicted Ranking: [6, 1, 0, 4, 2, 3, 5, 8, 7, 9]
  True Ranking:      [6, 1, 0, 4, 3, 2, 5, 8, 7, 9]
Sample 2:
  Predicted Ranking: [8, 9, 4, 5, 6, 1, 2, 0, 7, 3]
  True Ranking:      [8, 9, 6, 4, 5, 1, 0, 2, 7, 3]
Sample 3:
  Predicted Ranking: [3, 9, 6, 5, 7, 8, 4, 1, 2, 0]
  True Ranking:      [6, 9, 8, 3, 7, 5, 1, 4, 2, 0]
Sample 4:
  Predicted Ranking: [6, 1, 5, 3, 0, 7, 8, 9, 2, 4]
  True Ranking:      [7, 6, 0, 1, 5, 8, 3, 4, 9, 2]
Sample 5:
  Predicted Ranking: [8, 1, 4, 5, 9, 0, 6, 3, 7, 2]
  True Ranking:      [8, 1, 5, 6, 4, 0, 9, 3, 7, 2]
Sample 6:
  Predicted Ranking: [0, 6, 9, 2, 7, 4, 1, 8, 5, 3]
  True Ranking:      [2, 6, 0, 9, 1, 7, 8, 5, 3, 4]
Sample 7:
  Predicted Ranking: [1, 3, 7, 5, 0, 4, 9, 8, 6, 2]
  True Ranking:      [7, 3, 0, 1, 5, 6, 8, 9, 4, 2]
Sample 8:
  Predicted Ranking: [9, 2, 3, 1, 8, 6, 7, 4, 5, 0]
  True Ranking:      [2, 9, 0, 8, 3, 6, 4, 1, 7, 5]
Sample 9:
  Predicted Ranking: [7, 2, 0, 6, 9, 8, 3, 5, 4, 1]
  T